In [6]:
#how decorators work
#addeding a decorator funtion that takes the base funtion as an argument
def add_sprinkles(func):
    def wrapper(*args, **kwargs):#definnig a wrapper function tha takes in any number of arguments
        print("Adds sprinkles... ")
        func(*args, **kwargs)#calling the base funtion
    return wrapper#returning the wrapper funtion ie the base function and the extended functionality/decorator

def add_fudge(func):
    def wrapper(*args, **kwargs):
        print("Adds fudge...")
        func(*args, **kwargs)
    return wrapper
    
#defining a base funtion
@add_fudge#calling the decorator function
@add_sprinkles
def print_ice_cream(flavor):
    print(f"Here is a {flavor} ice cream")

print_ice_cream("vanilla")#calling the base funtion

Adds fudge...
Adds sprinkles... 
Here is a vanilla ice cream


In [1]:
def my_map(my_func, list):
    result = []
    for item in list:
        new_item = my_func(item)
        result.append(new_item)
    return result


num = [3, 4, 5 , 6, 7]
print(my_map((lambda x: x**3), num))

[27, 64, 125, 216, 343]


In [ ]:
from experta import *
from collections import defaultdict

class Rule:
    def __init__(self, *args):
        self.conditions = []
        for arg in args:
            if isinstance(arg, Fact):
                self.conditions.append(arg)
            elif callable(arg):
                self.conditions.append(arg)

    def __call__(self, engine):
        """
        This method is called when the rule is evaluated.
        It checks if all conditions are met and executes the associated action.
        """
        for condition in self.conditions:
            if callable(condition):
                if not condition(engine):
                    return False  # Condition not met
            elif not engine.facts.match(condition):
                return False  # Fact not found

        # All conditions met, execute the action
        engine.agenda.append(self)
        return True

class KnowledgeEngine:
    def __init__(self):
        self.facts = defaultdict(list)
        self.agenda = []

    def declare(self, fact):
        if isinstance(fact, Fact):
            self.facts[type(fact)].append(fact)
        else:
            raise TypeError("Only Fact objects can be declared.")

    def run(self):
        while self.agenda:
            rule = self.agenda.pop(0)
            rule.execute(self) 

# Example Usage
class MyKnowledgeBase(KnowledgeEngine):
    @Rule(Fact('animal', family='felinae'))
    def is_a_cat(self):
        print("This is a cat!")

engine = MyKnowledgeBase()
engine.reset()
engine.declare(Fact('animal', family='felinae'))
engine.run()

This is a cat!
This is a cat!


In [ ]:
from experta import *

class diagoseEngine(KnowledgeEngine):
    @Rule(Fact(symptom ='fever'))
    def malaria(self):
        print("You have malaria")

    @Rule(Fact(symptom = 'cough'))
    def flu(self):
        print("You have a flu")
    
    @Rule(Fact(symptom = 'headache'))
    def hypertension(self):
        print("you have hypertension")
    
    #adding a wildcard rule
    @Rule(NOT(Fact(symptom = W())))
    def dont_know(self):
        print("I dont know what you have")  
    

engine = diagoseEngine()
engine.reset()

symptoms = input("enter a list of your symptoms comma seperated")
engineInput = symptoms.lower().split(",")
for s in engineInput:
    engine.declare(Fact(symptom = s))

engine.run()cough_

In [15]:
class KnowledgeBase:
    def __init__(self):
        self.diseases = {
            "Common Cold": {"symptoms": ["cough", "sneezing", "sore throat"], "treatment": "rest and hydration"},
            "Flu": {"symptoms": ["fever", "fatigue", "body aches"], "treatment": "antiviral medication"},
            "Pneumonia": {"symptoms": ["chest pain", "cough", "fever"], "treatment": "antibiotics"},
            "Tuberculosis": {"symptoms": ["cough with blood", "fatigue", "weight loss"], "treatment": "antibiotics"},
            "Allergies": {"symptoms": ["sneezing", "itchy eyes", "rash"], "treatment": "antihistamines"}
        }

        self.relationships = {
            "Common Cold": ["Flu", "Allergies"],
            "Flu": ["Common Cold", "Pneumonia"],
            "Pneumonia": ["Flu"],
            "Tuberculosis": [],
            "Allergies": ["Common Cold"],
            "cough": ["Common Cold", "Flu", "Pneumonia", "Tuberculosis"],
            "sneezing": ["Common Cold", "Flu", "Allergies"],
            "fever": ["Flu", "Pneumonia"]
        }

    def diagnose(self, symptoms):
        possible_diseases = {}
        for symptom in symptoms:
            for disease in self.relationships.get(symptom, []):
                possible_diseases[disease] = possible_diseases.get(disease, 0) + 1

        # Sort diseases by the number of matching symptoms (most relevant first)
        sorted_diseases = sorted(possible_diseases.items(), key=lambda x: x[1], reverse=True)

        return [disease for disease, _ in sorted_diseases]

# Interactive mode
kb = KnowledgeBase()
print("Welcome to the Medical Diagnosis Knowledge-Based System")
print("Possible symptoms:", ", ".join(kb.relationships.keys()))
print("\nEnter symptoms (comma separated):")
user_input = input(">").lower().split(",")

symptoms = [symptom.strip() for symptom in user_input]
diagnoses = kb.diagnose(symptoms)

if diagnoses:
    print("Possible diagnoses:")
    for disease in diagnoses:
        print(f"- {disease}: {kb.diseases[disease]['treatment']}")
else:
    print("No diagnoses found. Please consult a doctor.")

Welcome to the Medical Diagnosis Knowledge-Based System
Possible symptoms: Common Cold, Flu, Pneumonia, Tuberculosis, Allergies, cough, sneezing, fever

Enter symptoms (comma separated):
No diagnoses found. Please consult a doctor.


In [ ]:
#practicing making a matchmaking system
class knowledgeBase:
    def __init__(self):
        self.profiles = {}
        self.next_id = 1

    def add_profile(self, profile_data):
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def get_all_profiles(self):
        return list(self.profiles.values())
    

    def displayData(self):
        print([{key: values} for key, values in self.profiles.items()])

class matchMaking:
    def __init__(self, knowledgeBase, weights, maxAgeDiff = 20):
        self.knowledgeBase = knowledgeBase
        self.weights = weights or {'age': 0.2, 'interest': 0.3, 'location': 0.5}
        self.maxAge = maxAgeDiff

    def find_compatibity_helper_function(self, profile_a, profile_b):
        #finding age compatibility
       age_diff = abs( profile_a['age']-profile_b['age'] )
       age_compatibity = max(0, 1- (age_diff/self.maxAge))

       #finding interest compatibility
       profile_a_interests = profile_a['interests']
       profile_b_interests = profile_b['interests']
       common_interests = len(profile_a_interests & profile_b_interests)
       all_interests = len(profile_a_interests | profile_b_interests)
       interest_compatibilty = (common_interests/all_interests) * self.weights['interest']

       #finding the location compatibilty
       location_compatibilty = (self.weights['location'] if profile_a['location'] == profile_b['location'] else 0)

       return age_compatibity + interest_compatibilty + location_compatibilty
    
    def matchmaker(self, user_id):
        profile_compatibilty_scores = {}
        target_profile = self.knowledgeBase.get('user_id')
        for profile in self.knowledgeBase.get_all_profiles():
            if profile != target_profile:
                profile_compatibilty_scores[profile['profile_id']] = self.find_compatibity_helper_function(target_profile, profile)

        sorted_profiles = sorted(self.knowledgeBase.items(), key= lambda x: x[1], reverse= True)
        return sorted_profiles
            






if __name__ == '__main__':
    #initialize the knowledgebase
    kb = knowledgeBase()

    #populate with sample profiles
    profiles = [
        {'age': 25, 'interests': ['hiking', 'biking'], 'location': 'NY'},
        {'age': 30, 'interests': ['biking', 'reading'], 'location': 'NY'},
        {'age': 28, 'interests': ['biking', 'movies'], 'location': 'CA'},
        {'age': 27, 'interests': ['hiking', 'movies'], 'location': 'CA'},
        {'age': 26, 'interests': ['reading', 'movies'], 'location': 'NY'}
    ]
    for p in profiles:
        kb.add_profile(p)
    
    kb.displayData()





[{1: {'id': 1, 'age': 25, 'interests': ['hiking', 'biking'], 'location': 'NY'}}, {2: {'id': 2, 'age': 30, 'interests': ['biking', 'reading'], 'location': 'NY'}}, {3: {'id': 3, 'age': 28, 'interests': ['biking', 'movies'], 'location': 'CA'}}, {4: {'id': 4, 'age': 27, 'interests': ['hiking', 'movies'], 'location': 'CA'}}, {5: {'id': 5, 'age': 26, 'interests': ['reading', 'movies'], 'location': 'NY'}}]


## practicing creating and simulating agents moving in 2D space

In [18]:
#importing usefull libraries
import matplotlib as plt
import random

In [19]:
#creating an agent class
class Agent:
    def __init__(self, x = None, y = None, grid_size = 100):
        self.grid_size = grid_size
        self.x = x if not None else random.randint(0, grid_size -1)
        self.y = y if not None else random.randint(0, grid_size -1)
    
    def move(self):
        #this moves randomly
        self.x = (self.x + random.choice([-1, 1])) % self.grid_size
        self.y = (self.y + random.choice([-1, 1])) % self.grid_size

In [20]:
#creating multiple agents
def Agents(n_agents, grid_size = 100):
    return[Agent(grid_size=grid_size) for _ in range(n_agents)]

In [21]:
#updating agent movements
def update(agents):
    for agent in agents:
        agent.move()

In [22]:
#visualizing the movement
def plot_agents(Agents, frames, grid_size=100):
    plt.figure(figsize = (6, 6))
    plt.xlim(0, grid_size)
    plt.ylim(0, grid_size)

    x_vals = [agent.x for agent in Agents]
    y_vals = [agent.y for agent in Agents]

    plt.scatter(x_vals, y_vals, color = "blue")
    plt.title("Agent Movents")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid(True)
    plt.show()

In [23]:
#running simulaiton
def run_the_simulation(n_agents=10, frames=10, grid_size=100):
    agents = Agents(n_agents, grid_size)

    for frame in range(frames):
        update(agents)
        plot_agents(agents, frame, grid_size)

In [24]:
run_the_simulation(5, 5, 100)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'